In [427]:

# SEAG qualification period is 22 Oct 2024 to 5 Sept 2025

#1. Segregate into Male and Femal 
#2. For each gender perform the following: 
#a. Sort data by mapped eent, then perf scalar (higher the better)
#b. Identify tiers based on performance - Tier 1 is meets bronze medal mark for SEAG, Tier 2 is 2% and Tier 3 is 3.5%
#c. Check - if athlete met bronze or 2%/3.5% then delta_benchmark is zero or +, delta2% is + and delta 3.5% is +
#d. Top ranked athletes for each event are chosen. Max number of athletes for each event is 3, except for 100m/400m which is 6
#    This includes athletes on spex scholarship and potential
#e. The max for each tier is 2. Lower ranked athletes move down one tier.
#3. If athlete qualifies for more than one event the higher tier event is given
#4. Jump and throws junior program to be solved separately

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [865]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic
import dateutil.parser as parser 
import datacompy
import pytz

from itertools import permutations



from google.cloud import storage



In [866]:
'''
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION, TIMESTAMP
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)


'''

'\n# PRODUCTION ENVIRONMENT\n# Extract timed event records\n\nimport pandas_gbq\nfrom google.oauth2 import service_account\n\ncredentials = service_account.Credentials.from_service_account_file(\n    \'/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json\',\n    \n    \n)\n\nsql1="""\nSELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION, TIMESTAMP\nFROM `saa-analytics.results.athlete_results_prod` \nWHERE RESULT!=\'NM\' AND RESULT!=\'-\' AND RESULT!=\'DNS\' AND RESULT!=\'DNF\' AND RESULT!=\'DNQ\' AND RESULT!=\'DQ\' AND RESULT IS NOT NULL\n\n"""\n\ncompetitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)\n\n\n'

## Load Data From BigQuery

In [1250]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION, TIMESTAMP
FROM `saa-analytics.results.PRODUCTION` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='FOUL' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|█████████████████████████████████████████████████████████████|


In [1980]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month
0,SERENA TEOH,38:38:00,INDIVIDUAL,36,,,"10,000m",,,,...,Female,Asics META Time Trial,2023-11-19,2023,International,2025-08-10 17:19:00+00:00,2025-08-16 17:26:50.424539+00:00,636 days 17:26:50.424539,636,11
1,"SOUNDARARAJAH, JEEVANEESH",33:54:00,INDIVIDUAL,30,,,"10,000m",,,,...,Male,Asics META Time Trial,2023-11-19,2023,International,2025-08-10 17:19:00+00:00,2025-08-16 17:26:50.424539+00:00,636 days 17:26:50.424539,636,11
2,"GOH, CHUI LING",39:57:00,INDIVIDUAL,31,,,"10,000m",,,,...,Female,Asics META Time Trial,2023-11-19,2023,International,2025-08-10 17:19:00+00:00,2025-08-16 17:26:50.424539+00:00,636 days 17:26:50.424539,636,11
3,"ABDUL RAZAK, ABDUL RAQIB",23.9,Singapore Sports School,17,1.0,,200m,,,,...,Male,Melacca Open,2023-11-19,2023,International,2025-08-10 17:19:00+00:00,2025-08-16 17:26:50.424539+00:00,636 days 17:26:50.424539,636,11
4,"ABDUL RAZAK, ABDUL RAQIB",24.7,Singapore Sports School,17,7.0,,200m,,,,...,Male,Melacca Open,2023-11-19,2023,International,2025-08-10 17:19:00+00:00,2025-08-16 17:26:50.424539+00:00,636 days 17:26:50.424539,636,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166859,Rachel See,1:22:07,Individual,40,1,nan,Half Marathon,,,,...,Female,Great Eastern Womens Run,2022-11-20,2022,Local,2025-08-15 16:07:00+00:00,2025-08-16 17:26:50.424539+00:00,1000 days 17:26:50.424539,1000,11
166860,LIM KIEN MAU,2:36:01,Individual,42,nan,nan,Marathon,,,,...,Male,Abu Dhabi Marathon,2022-12-17,2022,International,2025-08-15 16:07:00+00:00,2025-08-16 17:26:50.424539+00:00,973 days 17:26:50.424539,973,12
166861,SHARON TAN FANG YU,1:24:46,Individual,33,nan,nan,Half Marathon,,,,...,Female,Melbourne Sandy Point Half Marathon,2022-08-27,2022,International,2025-08-15 16:07:00+00:00,2025-08-16 17:26:50.424539+00:00,1085 days 17:26:50.424539,1085,8
166862,DEVATHAS SATIANATHAN,2:49:34,Individual,34,nan,nan,Marathon,,,,...,Male,Chicago Marathon,2022-10-09,2022,International,2025-08-15 16:07:00+00:00,2025-08-16 17:26:50.424539+00:00,1042 days 17:26:50.424539,1042,10


In [1252]:
competitors.drop_duplicates(inplace=True)


In [1253]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
0,SERENA TEOH,38:38:00,INDIVIDUAL,36,,,"10,000m",,,,1987,,-,Long,Female,Asics META Time Trial,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00
1,"SOUNDARARAJAH, JEEVANEESH",33:54:00,INDIVIDUAL,30,,,"10,000m",,,,1993,,-,Long,Male,Asics META Time Trial,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00
2,"GOH, CHUI LING",39:57:00,INDIVIDUAL,31,,,"10,000m",,,,1992,,-,Long,Female,Asics META Time Trial,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00
3,"ABDUL RAZAK, ABDUL RAQIB",23.9,Singapore Sports School,17,1.0,,200m,,,,2006,,HT,Sprint,Male,Melacca Open,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00
4,"ABDUL RAZAK, ABDUL RAQIB",24.7,Singapore Sports School,17,7.0,,200m,,,,2006,,HT,Sprint,Male,Melacca Open,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166859,Rachel See,1:22:07,Individual,40,1,nan,Half Marathon,,,,1982,,-,Marathon,Female,Great Eastern Womens Run,2022-11-20 00:00:00+00:00,2022,Local,2025-08-15 16:07:00+00:00
166860,LIM KIEN MAU,2:36:01,Individual,42,nan,nan,Marathon,,,,1981,,-,Marathon,Male,Abu Dhabi Marathon,2022-12-17 00:00:00+00:00,2022,International,2025-08-15 16:07:00+00:00
166861,SHARON TAN FANG YU,1:24:46,Individual,33,nan,nan,Half Marathon,,,,1990,,-,Marathon,Female,Melbourne Sandy Point Half Marathon,2022-08-27 00:00:00+00:00,2022,International,2025-08-15 16:07:00+00:00
166862,DEVATHAS SATIANATHAN,2:49:34,Individual,34,nan,nan,Marathon,,,,1988,,-,Marathon,Male,Chicago Marathon,2022-10-09 00:00:00+00:00,2022,International,2025-08-15 16:07:00+00:00


In [1254]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Throws Program/')


competitors.to_csv('athletes_all.csv', sep=',', encoding='utf-8-sig', index=False)

In [1255]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP
0,SERENA TEOH,38:38:00,INDIVIDUAL,36,,,"10,000m",,,,1987,,-,Long,Female,Asics META Time Trial,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00
1,"SOUNDARARAJAH, JEEVANEESH",33:54:00,INDIVIDUAL,30,,,"10,000m",,,,1993,,-,Long,Male,Asics META Time Trial,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00
2,"GOH, CHUI LING",39:57:00,INDIVIDUAL,31,,,"10,000m",,,,1992,,-,Long,Female,Asics META Time Trial,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00
3,"ABDUL RAZAK, ABDUL RAQIB",23.9,Singapore Sports School,17,1.0,,200m,,,,2006,,HT,Sprint,Male,Melacca Open,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00
4,"ABDUL RAZAK, ABDUL RAQIB",24.7,Singapore Sports School,17,7.0,,200m,,,,2006,,HT,Sprint,Male,Melacca Open,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166859,Rachel See,1:22:07,Individual,40,1,nan,Half Marathon,,,,1982,,-,Marathon,Female,Great Eastern Womens Run,2022-11-20 00:00:00+00:00,2022,Local,2025-08-15 16:07:00+00:00
166860,LIM KIEN MAU,2:36:01,Individual,42,nan,nan,Marathon,,,,1981,,-,Marathon,Male,Abu Dhabi Marathon,2022-12-17 00:00:00+00:00,2022,International,2025-08-15 16:07:00+00:00
166861,SHARON TAN FANG YU,1:24:46,Individual,33,nan,nan,Half Marathon,,,,1990,,-,Marathon,Female,Melbourne Sandy Point Half Marathon,2022-08-27 00:00:00+00:00,2022,International,2025-08-15 16:07:00+00:00
166862,DEVATHAS SATIANATHAN,2:49:34,Individual,34,nan,nan,Marathon,,,,1988,,-,Marathon,Male,Chicago Marathon,2022-10-09 00:00:00+00:00,2022,International,2025-08-15 16:07:00+00:00


In [1256]:
# DATE column to contain timezone - tz aware mode

competitors['DATE'] = pd.to_datetime(competitors['DATE'], format='mixed', dayfirst=False, utc=True)


In [1257]:
# datetime to contain UTC (timezone)

competitors['NOW'] = datetime.datetime.now()

timezone = pytz.timezone('UTC')

competitors['NOW'] = datetime.datetime.now().replace(tzinfo=timezone)

In [1258]:
competitors['NOW']

0        2025-08-16 17:26:50.424539+00:00
1        2025-08-16 17:26:50.424539+00:00
2        2025-08-16 17:26:50.424539+00:00
3        2025-08-16 17:26:50.424539+00:00
4        2025-08-16 17:26:50.424539+00:00
                       ...               
166859   2025-08-16 17:26:50.424539+00:00
166860   2025-08-16 17:26:50.424539+00:00
166861   2025-08-16 17:26:50.424539+00:00
166862   2025-08-16 17:26:50.424539+00:00
166863   2025-08-16 17:26:50.424539+00:00
Name: NOW, Length: 163021, dtype: datetime64[us, UTC]

In [1259]:
# Calculate number of days from today to event date

#competitors['DATE'] = pd.to_datetime(competitors['DATE'], format='mixed', dayfirst=False, utc=False)

competitors['delta_time'] = competitors['NOW'] - competitors['DATE']


#competitors['delta_time'] = datetime.datetime.now() - competitors['DATE']


competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')

competitors['event_month'] = competitors['DATE'].dt.month


In [1260]:
competitors['delta_time']

0         636 days 17:26:50.424539
1         636 days 17:26:50.424539
2         636 days 17:26:50.424539
3         636 days 17:26:50.424539
4         636 days 17:26:50.424539
                    ...           
166859   1000 days 17:26:50.424539
166860    973 days 17:26:50.424539
166861   1085 days 17:26:50.424539
166862   1042 days 17:26:50.424539
166863   2065 days 17:26:50.424539
Name: delta_time, Length: 163021, dtype: timedelta64[ns]

In [1261]:
# Make sure date conversion is is valid for all rows

assert not competitors['delta_time'].isna().any()

In [1262]:
# These results have not had their event dates converted properly

competitors[competitors['DATE'].isna()]

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month


In [1263]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month
0,SERENA TEOH,38:38:00,INDIVIDUAL,36,,,"10,000m",,,,...,Female,Asics META Time Trial,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00,2025-08-16 17:26:50.424539+00:00,636 days 17:26:50.424539,636,11
1,"SOUNDARARAJAH, JEEVANEESH",33:54:00,INDIVIDUAL,30,,,"10,000m",,,,...,Male,Asics META Time Trial,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00,2025-08-16 17:26:50.424539+00:00,636 days 17:26:50.424539,636,11
2,"GOH, CHUI LING",39:57:00,INDIVIDUAL,31,,,"10,000m",,,,...,Female,Asics META Time Trial,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00,2025-08-16 17:26:50.424539+00:00,636 days 17:26:50.424539,636,11
3,"ABDUL RAZAK, ABDUL RAQIB",23.9,Singapore Sports School,17,1.0,,200m,,,,...,Male,Melacca Open,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00,2025-08-16 17:26:50.424539+00:00,636 days 17:26:50.424539,636,11
4,"ABDUL RAZAK, ABDUL RAQIB",24.7,Singapore Sports School,17,7.0,,200m,,,,...,Male,Melacca Open,2023-11-19 00:00:00+00:00,2023,International,2025-08-10 17:19:00+00:00,2025-08-16 17:26:50.424539+00:00,636 days 17:26:50.424539,636,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166859,Rachel See,1:22:07,Individual,40,1,nan,Half Marathon,,,,...,Female,Great Eastern Womens Run,2022-11-20 00:00:00+00:00,2022,Local,2025-08-15 16:07:00+00:00,2025-08-16 17:26:50.424539+00:00,1000 days 17:26:50.424539,1000,11
166860,LIM KIEN MAU,2:36:01,Individual,42,nan,nan,Marathon,,,,...,Male,Abu Dhabi Marathon,2022-12-17 00:00:00+00:00,2022,International,2025-08-15 16:07:00+00:00,2025-08-16 17:26:50.424539+00:00,973 days 17:26:50.424539,973,12
166861,SHARON TAN FANG YU,1:24:46,Individual,33,nan,nan,Half Marathon,,,,...,Female,Melbourne Sandy Point Half Marathon,2022-08-27 00:00:00+00:00,2022,International,2025-08-15 16:07:00+00:00,2025-08-16 17:26:50.424539+00:00,1085 days 17:26:50.424539,1085,8
166862,DEVATHAS SATIANATHAN,2:49:34,Individual,34,nan,nan,Marathon,,,,...,Male,Chicago Marathon,2022-10-09 00:00:00+00:00,2022,International,2025-08-15 16:07:00+00:00,2025-08-16 17:26:50.424539+00:00,1042 days 17:26:50.424539,1042,10


In [1264]:
# Choose date range window from 1 Jan 2024 to current


competitors['DATE']=competitors['DATE'].dt.tz_localize(None)  # switch off timezone for compatibility with np.datetime64

start = datetime.datetime(2024, 1, 1)
end = datetime.datetime(2025, 8, 7)

start_date = np.datetime64(start)
end_date = np.datetime64(end)


mask = (competitors['DATE'] >= start_date) & (competitors['DATE'] <= end_date)
athletes_selected = competitors.loc[mask]



In [1265]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Throws Program/')

athletes_selected.to_csv('athletes_downloaded_all.csv', encoding='utf-8')

In [1266]:
# Select all of 2024/25 for OCTC

#athletes_selected = competitors[(competitors['YEAR']=='2024')|(competitors['YEAR']=='2025')]

#athletes_selected = competitors[(competitors['YEAR']=='2025')]

In [1267]:
athletes_selected

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month
1505,Tedd Toh,10.97,,,3,,100m,,,,...,Male,PATAFA Weekly Relay Series - Seniors annd U20,2025-08-02,2025,International,2025-08-14 12:50:00+00:00,2025-08-16 17:26:50.424539+00:00,14 days 17:26:50.424539,14,8
9989,Tan Daryl,10.78,,,5,,100m,,,,...,Male,32nd Summer Universiade,2025-07-27,2025,International,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,20 days 17:26:50.424539,20,7
9990,Yan Teo,21.62,,,6,,200m,,,,...,Male,32nd Summer Universiade,2025-07-27,2025,International,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,20 days 17:26:50.424539,20,7
9991,Andrew George Medina,7.21,,,28,,Long Jump,,,,...,Male,32nd Summer Universiade,2025-07-27,2025,International,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,20 days 17:26:50.424539,20,7
9992,Laavina Jaganith,12.41,,,7,,100m,,,,...,Female,32nd Summer Universiade,2025-07-27,2025,International,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,20 days 17:26:50.424539,20,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58613,Tia Louise Rozario,12.78,nan,nan,nan,nan,Triple Jump,nan,nan,nan,...,Female,National Championships,2025-06-25,2025,International,2025-06-29 22:00:26.182128,2025-08-16 17:26:50.424539+00:00,52 days 17:26:50.424539,52,6
58614,"SYED AHMED RIADH, SHARIFAH FALISHA",11.13,SINGAPORE SPORTS SCHOOL,18,None,None,Triple jump,None,None,None,...,Female,FTKLAA State Meet,2024-03-03,2024,International,None,2025-08-16 17:26:50.424539+00:00,531 days 17:26:50.424539,531,3
58615,GOH YEN YOUNG AMELIA,10.73,SINGAPORE,16,2,U18,Triple jump,None,None,None,...,Female,Hong Kong Inter-City,2024-07-06,2024,International,None,2025-08-16 17:26:50.424539+00:00,406 days 17:26:50.424539,406,7
58616,LEE GABRIEL JIN YI,14.5,SINGAPORE,21,3,None,Triple jump,None,None,None,...,Male,Victor Saneev Memorial (Georgia),2024-10-26,2024,International,None,2025-08-16 17:26:50.424539+00:00,294 days 17:26:50.424539,294,10


In [2103]:
# Copy Source Dataframe

athletes = athletes_selected.copy()

In [2104]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month
1505,Tedd Toh,10.97,,,3,,100m,,,,...,Male,PATAFA Weekly Relay Series - Seniors annd U20,2025-08-02,2025,International,2025-08-14 12:50:00+00:00,2025-08-16 17:26:50.424539+00:00,14 days 17:26:50.424539,14,8
9989,Tan Daryl,10.78,,,5,,100m,,,,...,Male,32nd Summer Universiade,2025-07-27,2025,International,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,20 days 17:26:50.424539,20,7
9990,Yan Teo,21.62,,,6,,200m,,,,...,Male,32nd Summer Universiade,2025-07-27,2025,International,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,20 days 17:26:50.424539,20,7
9991,Andrew George Medina,7.21,,,28,,Long Jump,,,,...,Male,32nd Summer Universiade,2025-07-27,2025,International,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,20 days 17:26:50.424539,20,7
9992,Laavina Jaganith,12.41,,,7,,100m,,,,...,Female,32nd Summer Universiade,2025-07-27,2025,International,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,20 days 17:26:50.424539,20,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58613,Tia Louise Rozario,12.78,nan,nan,nan,nan,Triple Jump,nan,nan,nan,...,Female,National Championships,2025-06-25,2025,International,2025-06-29 22:00:26.182128,2025-08-16 17:26:50.424539+00:00,52 days 17:26:50.424539,52,6
58614,"SYED AHMED RIADH, SHARIFAH FALISHA",11.13,SINGAPORE SPORTS SCHOOL,18,None,None,Triple jump,None,None,None,...,Female,FTKLAA State Meet,2024-03-03,2024,International,None,2025-08-16 17:26:50.424539+00:00,531 days 17:26:50.424539,531,3
58615,GOH YEN YOUNG AMELIA,10.73,SINGAPORE,16,2,U18,Triple jump,None,None,None,...,Female,Hong Kong Inter-City,2024-07-06,2024,International,None,2025-08-16 17:26:50.424539+00:00,406 days 17:26:50.424539,406,7
58616,LEE GABRIEL JIN YI,14.5,SINGAPORE,21,3,None,Triple jump,None,None,None,...,Male,Victor Saneev Memorial (Georgia),2024-10-26,2024,International,None,2025-08-16 17:26:50.424539+00:00,294 days 17:26:50.424539,294,10


## Choose Throw Category and Find Missing DOBs

In [2105]:
# Filter out irrelevant events/classes and choose throws only category of events

athletes=athletes[athletes['CATEGORY_EVENT']=='Throw']

search_string='U11|U12|U13|U14|U15|300g|400g|Medicine|Turbo|Masters|F40/F42/F46/F63'

does_not_contain1 = ~athletes['EVENT_CLASS'].str.contains(search_string, na=False) 
does_not_contain2 = ~athletes['DIVISION'].str.contains(search_string, na=False) 
does_not_contain3 = ~athletes['RESULT'].str.contains('FOUL', na=False) 
does_not_contain4 = ~athletes['EVENT'].str.contains('Hammer Throw|Turbo', na=False) 
does_not_contain5 = ~athletes['COMPETITION'].str.contains('Para', na=False) 


athletes = athletes[does_not_contain1]
athletes = athletes[does_not_contain2]
athletes = athletes[does_not_contain3]
athletes = athletes[does_not_contain4]
athletes = athletes[does_not_contain5]



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/2187166660.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  athletes = athletes[does_not_contain2]
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/2187166660.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  athletes = athletes[does_not_contain3]
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/2187166660.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  athletes = athletes[does_not_contain4]
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/2187166660.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  athletes = athletes[does_not_contain5]


In [2106]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month
9995,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,Male,Kermismeeting,2025-07-20 00:00:00,2025,International,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,27 days 17:26:50.424539,27,7
10465,Eric Yee Chun Wei,45.15,,,,,Discus Throw,,,,...,Male,Johor Open Athletics Champs,2025-06-29 00:00:00,2025,International,2025-08-05 16:04:00+00:00,2025-08-16 17:26:50.424539+00:00,48 days 17:26:50.424539,48,6
10466,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,Male,Johor Open Athletics Champs,2025-06-29 00:00:00,2025,International,2025-08-05 16:04:00+00:00,2025-08-16 17:26:50.424539+00:00,48 days 17:26:50.424539,48,6
26160,Muhammad Armando Alatas Bin Mohd Zuraimi,6.13,Singapore Disability Sports Council -,,1,,Shot Put,,4Kilogram,,...,Male,Pesta Sukan Athletics 2025,2025-08-02 17:35:00,2025,Local,2025-08-15 14:46:00+00:00,2025-08-16 17:26:50.424539+00:00,13 days 23:51:50.424539,13,8
26161,Ahmad Afiq Ikhsan Bin Muhammad Yusman,5.30,Singapore Disability Sports Council -,,2,,Shot Put,,4Kilogram,,...,Male,Pesta Sukan Athletics 2025,2025-08-02 17:35:00,2025,Local,2025-08-15 14:46:00+00:00,2025-08-16 17:26:50.424539+00:00,13 days 23:51:50.424539,13,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57273,"Seu, Hoy Xuan Evelyn",9.25m,Singapore Sports School,15,2,U18,Shot Put,0.0,(3kg),nan,...,Female,SSP Invitational U15 and U18 2025,2025-05-31 00:00:00,2025,Local,2025-07-10 17:26:43.048429+00:00,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5
57274,"Ng, Reeyern",7.60m,Singapore Sports School,15,5,U18,Shot Put,0.0,(3kg),nan,...,Female,SSP Invitational U15 and U18 2025,2025-05-31 00:00:00,2025,Local,2025-07-10 17:26:43.048429+00:00,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5
57276,"Abdullah, Siti Annur Nadiera",10.45m,SSNTHO,17,1,U18,Shot Put,0.0,(3kg),nan,...,Female,SSP Invitational U15 and U18 2025,2025-05-31 00:00:00,2025,Local,2025-07-10 17:26:43.048429+00:00,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5
57277,"SHEIK ALAUDDIN, SHEIK FAYZ",10.51,SINGAPORE SPORTS SCHOOL,21,5,None,Shot put,None,7.26kg,None,...,Male,Perak Open,2024-05-10 00:00:00,2024,International,None,2025-08-16 17:26:50.424539+00:00,463 days 17:26:50.424539,463,5


In [2107]:
def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B






In [2108]:
# Process dates to extract age

# U18 end of 31 Dec 2025 below 18 (16 to 17)
# U20 end of 31 Dec 2025 below 20 (18 to 19)
# Senior(OPEN) end of 31 Dec 2025 (>=16 yrs)

# Map NSG divisions into age

mask = (athletes['DIVISION'].str.contains(r'A', na=False) & athletes['COMPETITION'].str.contains(r'National School Games', na=False))
athletes.loc[mask, 'AGE'] = '17.5'

mask = (athletes['DIVISION'].str.contains(r'B', na=False) & athletes['COMPETITION'].str.contains(r'National School Games', na=False))
athletes.loc[mask, 'AGE'] = '15.5'

mask = (athletes['DIVISION'].str.contains(r'C', na=False) & athletes['COMPETITION'].str.contains(r'National School Games', na=False))
athletes.loc[mask, 'AGE'] = '13.5'

mask = (athletes['DIVISION'].str.contains(r'O', na=False) & athletes['COMPETITION'].str.contains(r'National School Games', na=False))  # deprecate
athletes.loc[mask, 'AGE'] = '12'




In [2109]:
# Get list of names with no DOB

mask = ((athletes['DOB']=='None')|(athletes['DOB']=='nan'))

#mask = ((final_df['DOB'].isnull()))
#mask = final_df['DOB_new']=='NaT'

missing_names = athletes.loc[mask]

name_list = missing_names['NAME'].str.casefold().to_list()


In [2110]:
len(name_list)

553

## Lookup DOBs Based On Name Permutations

In [2111]:
# Strip out punctuation from NAME
from itertools import permutations
import string


#translator = str.maketrans('', '', string.punctuation)
#athletes['clean_name'] = athletes['NAME'].str.translate(translator)
#athletes['clean_name'] = athletes['clean_name'].str.casefold()


athletes = athletes.reset_index(drop=True)

translator = str.maketrans('', '', string.punctuation)

athletes['clean_name'] = athletes['NAME'].apply(lambda x: str(x).translate(translator))
athletes['clean_name'] = athletes['clean_name'].str.casefold()


In [2112]:
# Create a list of name components to seach for permutations

athletes['name_to_list'] = athletes['clean_name'].str.split(' ')


athletes


,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DATE,YEAR,REGION,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month,clean_name,name_to_list
0,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,2025-07-20 00:00:00,2025,International,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,27 days 17:26:50.424539,27,7,eric yee chun wai,"[eric, yee, chun, wai]"
1,Eric Yee Chun Wei,45.15,,,,,Discus Throw,,,,...,2025-06-29 00:00:00,2025,International,2025-08-05 16:04:00+00:00,2025-08-16 17:26:50.424539+00:00,48 days 17:26:50.424539,48,6,eric yee chun wei,"[eric, yee, chun, wei]"
2,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,2025-06-29 00:00:00,2025,International,2025-08-05 16:04:00+00:00,2025-08-16 17:26:50.424539+00:00,48 days 17:26:50.424539,48,6,lucas wong yijie,"[lucas, wong, yijie]"
3,Muhammad Armando Alatas Bin Mohd Zuraimi,6.13,Singapore Disability Sports Council -,,1,,Shot Put,,4Kilogram,,...,2025-08-02 17:35:00,2025,Local,2025-08-15 14:46:00+00:00,2025-08-16 17:26:50.424539+00:00,13 days 23:51:50.424539,13,8,muhammad armando alatas bin mohd zuraimi,"[muhammad, armando, alatas, bin, mohd, zuraimi]"
4,Ahmad Afiq Ikhsan Bin Muhammad Yusman,5.30,Singapore Disability Sports Council -,,2,,Shot Put,,4Kilogram,,...,2025-08-02 17:35:00,2025,Local,2025-08-15 14:46:00+00:00,2025-08-16 17:26:50.424539+00:00,13 days 23:51:50.424539,13,8,ahmad afiq ikhsan bin muhammad yusman,"[ahmad, afiq, ikhsan, bin, muhammad, yusman]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,"Seu, Hoy Xuan Evelyn",9.25m,Singapore Sports School,15,2,U18,Shot Put,0.0,(3kg),nan,...,2025-05-31 00:00:00,2025,Local,2025-07-10 17:26:43.048429+00:00,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,seu hoy xuan evelyn,"[seu, hoy, xuan, evelyn]"
1686,"Ng, Reeyern",7.60m,Singapore Sports School,15,5,U18,Shot Put,0.0,(3kg),nan,...,2025-05-31 00:00:00,2025,Local,2025-07-10 17:26:43.048429+00:00,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,ng reeyern,"[ng, reeyern]"
1687,"Abdullah, Siti Annur Nadiera",10.45m,SSNTHO,17,1,U18,Shot Put,0.0,(3kg),nan,...,2025-05-31 00:00:00,2025,Local,2025-07-10 17:26:43.048429+00:00,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,abdullah siti annur nadiera,"[abdullah, siti, annur, nadiera]"
1688,"SHEIK ALAUDDIN, SHEIK FAYZ",10.51,SINGAPORE SPORTS SCHOOL,21,5,None,Shot put,None,7.26kg,None,...,2024-05-10 00:00:00,2024,International,None,2025-08-16 17:26:50.424539+00:00,463 days 17:26:50.424539,463,5,sheik alauddin sheik fayz,"[sheik, alauddin, sheik, fayz]"


In [2113]:

athletes['dob_list'] = None  


for index, row in athletes.iterrows():
         
   # if str(row['DOB_new']) == 'NaT':
   # if ((final_df['DOB']=='None') or (final_df['DOB']=='nan')):
            
    name_components = row['name_to_list']
        
    permutations_list = []
    dob_list=[]
    string=''
    
    
    for p in permutations(name_components):
        
        string = " ".join(p)
        
        permutations_list.append(string)

        dob = dictionary_dob_clean_name.get(string)
        
        if dob is not None:
            dob_list.append(dob)

    
  #  null.at[index, 'permutations'] = permutations_list
   # athletes.loc[index, 'dob_list'] = dob_list
    athletes.at[index, 'dob_list'] = [dob_list] if isinstance(dob_list, list) else dob_list




In [2114]:
from itertools import permutations

# Make sure the target columns exist
athletes['permutations'] = None
athletes['dob_list'] = None

for index, row in athletes.iterrows():
    name_components = row['name_to_list']
    
    # If the row has no valid components, skip
    if not isinstance(name_components, (list, tuple)) or len(name_components) == 0:
        athletes.at[index, 'permutations'] = []
        athletes.at[index, 'dob_list'] = []
        continue
    
    permutations_list = []
    dob_list = []
    
    for p in permutations(name_components):
        string = " ".join(p)
        permutations_list.append(string)

        dob = dictionary_dob_clean_name.get(string)
        if dob is not None:
            dob_list.append(dob)

    # Save lists directly into cells (as Python objects)
    athletes.at[index, 'permutations'] = permutations_list
    athletes.at[index, 'dob_list'] = dob_list


In [2115]:
# Create a new column with the first element of dob_list (or NaN if empty)
athletes['dob_first'] = athletes['dob_list'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)


In [2116]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month,clean_name,name_to_list,dob_list,permutations,dob_first
0,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,2025-07-25 17:36:00+00:00,2025-08-16 17:26:50.424539+00:00,27 days 17:26:50.424539,27,7,eric yee chun wai,"[eric, yee, chun, wai]","[1999-03-01 00:00:00, 1999-03-01 00:00:00, 199...","[eric yee chun wai, eric yee wai chun, eric ch...",1999-03-01
1,Eric Yee Chun Wei,45.15,,,,,Discus Throw,,,,...,2025-08-05 16:04:00+00:00,2025-08-16 17:26:50.424539+00:00,48 days 17:26:50.424539,48,6,eric yee chun wei,"[eric, yee, chun, wei]",[],"[eric yee chun wei, eric yee wei chun, eric ch...",NaT
2,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,2025-08-05 16:04:00+00:00,2025-08-16 17:26:50.424539+00:00,48 days 17:26:50.424539,48,6,lucas wong yijie,"[lucas, wong, yijie]",[2004-05-05 00:00:00],"[lucas wong yijie, lucas yijie wong, wong luca...",2004-05-05
3,Muhammad Armando Alatas Bin Mohd Zuraimi,6.13,Singapore Disability Sports Council -,,1,,Shot Put,,4Kilogram,,...,2025-08-15 14:46:00+00:00,2025-08-16 17:26:50.424539+00:00,13 days 23:51:50.424539,13,8,muhammad armando alatas bin mohd zuraimi,"[muhammad, armando, alatas, bin, mohd, zuraimi]",[2011-03-23 00:00:00],"[muhammad armando alatas bin mohd zuraimi, muh...",2011-03-23
4,Ahmad Afiq Ikhsan Bin Muhammad Yusman,5.30,Singapore Disability Sports Council -,,2,,Shot Put,,4Kilogram,,...,2025-08-15 14:46:00+00:00,2025-08-16 17:26:50.424539+00:00,13 days 23:51:50.424539,13,8,ahmad afiq ikhsan bin muhammad yusman,"[ahmad, afiq, ikhsan, bin, muhammad, yusman]",[2011-03-10 00:00:00],"[ahmad afiq ikhsan bin muhammad yusman, ahmad ...",2011-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,"Seu, Hoy Xuan Evelyn",9.25m,Singapore Sports School,15,2,U18,Shot Put,0.0,(3kg),nan,...,2025-07-10 17:26:43.048429+00:00,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,seu hoy xuan evelyn,"[seu, hoy, xuan, evelyn]",[2010-10-19 00:00:00],"[seu hoy xuan evelyn, seu hoy evelyn xuan, seu...",2010-10-19
1686,"Ng, Reeyern",7.60m,Singapore Sports School,15,5,U18,Shot Put,0.0,(3kg),nan,...,2025-07-10 17:26:43.048429+00:00,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,ng reeyern,"[ng, reeyern]",[2010-03-23 00:00:00],"[ng reeyern, reeyern ng]",2010-03-23
1687,"Abdullah, Siti Annur Nadiera",10.45m,SSNTHO,17,1,U18,Shot Put,0.0,(3kg),nan,...,2025-07-10 17:26:43.048429+00:00,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,abdullah siti annur nadiera,"[abdullah, siti, annur, nadiera]",[2008-01-01 00:00:00],"[abdullah siti annur nadiera, abdullah siti na...",2008-01-01
1688,"SHEIK ALAUDDIN, SHEIK FAYZ",10.51,SINGAPORE SPORTS SCHOOL,21,5,None,Shot put,None,7.26kg,None,...,None,2025-08-16 17:26:50.424539+00:00,463 days 17:26:50.424539,463,5,sheik alauddin sheik fayz,"[sheik, alauddin, sheik, fayz]","[2003-08-16 00:00:00, 2003-08-16 00:00:00]","[sheik alauddin sheik fayz, sheik alauddin fay...",2003-08-16


In [2117]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Throws Program/')

athletes.to_csv('final.csv', encoding='utf-8')

In [2118]:
# show names without a match

null = athletes[athletes['dob_first'].isnull()]

In [2119]:
null

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,TIMESTAMP,NOW,delta_time,delta_time_conv,event_month,clean_name,name_to_list,dob_list,permutations,dob_first
1,Eric Yee Chun Wei,45.15,,,,,Discus Throw,,,,...,2025-08-05 16:04:00+00:00,2025-08-16 17:26:50.424539+00:00,48 days 17:26:50.424539,48,6,eric yee chun wei,"[eric, yee, chun, wei]",[],"[eric yee chun wei, eric yee wei chun, eric ch...",NaT
203,Poh Yan An Darius,32.01,RI,15.5,8,B,Discus,None,1.50kg,None,...,None,2025-08-16 17:26:50.424539+00:00,498 days 17:26:50.424539,498,4,poh yan an darius,"[poh, yan, an, darius]",[],"[poh yan an darius, poh yan darius an, poh an ...",NaT
204,Jayne Allyson Schultz Tan,20.20,CG,13.5,2,C,Discus,None,1.00kg,None,...,None,2025-08-16 17:26:50.424539+00:00,498 days 17:26:50.424539,498,4,jayne allyson schultz tan,"[jayne, allyson, schultz, tan]",[],"[jayne allyson schultz tan, jayne allyson tan ...",NaT
205,Leroy Andre Chng Chong Kai,41.20,VJC,17.5,3,A,Discus,None,1.50kg,None,...,None,2025-08-16 17:26:50.424539+00:00,498 days 17:26:50.424539,498,4,leroy andre chng chong kai,"[leroy, andre, chng, chong, kai]",[],"[leroy andre chng chong kai, leroy andre chng ...",NaT
206,Dylan Seah (Xie Kaijie),40.44,HCI,15.5,5,B,Discus,None,1.50kg,None,...,None,2025-08-16 17:26:50.424539+00:00,498 days 17:26:50.424539,498,4,dylan seah xie kaijie,"[dylan, seah, xie, kaijie]",[],"[dylan seah xie kaijie, dylan seah kaijie xie,...",NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665,Sim En Qi,7.86,BTV,15.5,13.0,B,Shot Put,None,3.00kg,None,...,2025-06-06 15:35:30.807106,2025-08-16 17:26:50.424539+00:00,113 days 17:26:50.424539,113,4,sim en qi,"[sim, en, qi]",[],"[sim en qi, sim qi en, en sim qi, en qi sim, q...",NaT
1666,Kendra Gomez,8.60,AIS,15.5,9.0,B,Shot Put,None,3.00kg,None,...,2025-06-06 15:35:30.807106,2025-08-16 17:26:50.424539+00:00,113 days 17:26:50.424539,113,4,kendra gomez,"[kendra, gomez]",[],"[kendra gomez, gomez kendra]",NaT
1667,Jayden Andre Lam Jia Long,13.86,SJI,17.5,7.0,A,Shot Put,None,5.00kg,None,...,2025-06-06 15:35:30.807106,2025-08-16 17:26:50.424539+00:00,113 days 17:26:50.424539,113,4,jayden andre lam jia long,"[jayden, andre, lam, jia, long]",[],"[jayden andre lam jia long, jayden andre lam l...",NaT
1668,"Lee Xing Hui, Rachel (Li Xinghui)",9.91,SNG,15.5,4.0,B,Shot Put,None,3.00kg,None,...,2025-06-06 15:35:30.807106,2025-08-16 17:26:50.424539+00:00,113 days 17:26:50.424539,113,4,lee xing hui rachel li xinghui,"[lee, xing, hui, rachel, li, xinghui]",[],"[lee xing hui rachel li xinghui, lee xing hui ...",NaT


In [2120]:
# Determine exact age of athletes

end = pd.Timestamp('2025-12-31')

# 2) Force DOB column to naive datetime (cast to string first to strip any tz info)
dob = pd.to_datetime(athletes['dob_first'].astype(str), errors='coerce')

# 3) Compute age in years (days / 365.25)
athletes['age_dec25'] = (end - dob).dt.days / 365.25

# 4) Filter: >19 and <20 years old on 2025-12-31
#mask = (age_years > 19) & (age_years < 20)
#athletes = athletes.loc[mask].copy()
#athletes['age_years'] = age_years[mask]


In [2121]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,NOW,delta_time,delta_time_conv,event_month,clean_name,name_to_list,dob_list,permutations,dob_first,age_dec25
0,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,2025-08-16 17:26:50.424539+00:00,27 days 17:26:50.424539,27,7,eric yee chun wai,"[eric, yee, chun, wai]","[1999-03-01 00:00:00, 1999-03-01 00:00:00, 199...","[eric yee chun wai, eric yee wai chun, eric ch...",1999-03-01,26.836413
1,Eric Yee Chun Wei,45.15,,,,,Discus Throw,,,,...,2025-08-16 17:26:50.424539+00:00,48 days 17:26:50.424539,48,6,eric yee chun wei,"[eric, yee, chun, wei]",[],"[eric yee chun wei, eric yee wei chun, eric ch...",NaT,NaN
2,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,2025-08-16 17:26:50.424539+00:00,48 days 17:26:50.424539,48,6,lucas wong yijie,"[lucas, wong, yijie]",[2004-05-05 00:00:00],"[lucas wong yijie, lucas yijie wong, wong luca...",2004-05-05,21.656400
3,Muhammad Armando Alatas Bin Mohd Zuraimi,6.13,Singapore Disability Sports Council -,,1,,Shot Put,,4Kilogram,,...,2025-08-16 17:26:50.424539+00:00,13 days 23:51:50.424539,13,8,muhammad armando alatas bin mohd zuraimi,"[muhammad, armando, alatas, bin, mohd, zuraimi]",[2011-03-23 00:00:00],"[muhammad armando alatas bin mohd zuraimi, muh...",2011-03-23,14.776181
4,Ahmad Afiq Ikhsan Bin Muhammad Yusman,5.30,Singapore Disability Sports Council -,,2,,Shot Put,,4Kilogram,,...,2025-08-16 17:26:50.424539+00:00,13 days 23:51:50.424539,13,8,ahmad afiq ikhsan bin muhammad yusman,"[ahmad, afiq, ikhsan, bin, muhammad, yusman]",[2011-03-10 00:00:00],"[ahmad afiq ikhsan bin muhammad yusman, ahmad ...",2011-03-10,14.811773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685,"Seu, Hoy Xuan Evelyn",9.25m,Singapore Sports School,15,2,U18,Shot Put,0.0,(3kg),nan,...,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,seu hoy xuan evelyn,"[seu, hoy, xuan, evelyn]",[2010-10-19 00:00:00],"[seu hoy xuan evelyn, seu hoy evelyn xuan, seu...",2010-10-19,15.200548
1686,"Ng, Reeyern",7.60m,Singapore Sports School,15,5,U18,Shot Put,0.0,(3kg),nan,...,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,ng reeyern,"[ng, reeyern]",[2010-03-23 00:00:00],"[ng reeyern, reeyern ng]",2010-03-23,15.775496
1687,"Abdullah, Siti Annur Nadiera",10.45m,SSNTHO,17,1,U18,Shot Put,0.0,(3kg),nan,...,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,abdullah siti annur nadiera,"[abdullah, siti, annur, nadiera]",[2008-01-01 00:00:00],"[abdullah siti annur nadiera, abdullah siti na...",2008-01-01,17.998631
1688,"SHEIK ALAUDDIN, SHEIK FAYZ",10.51,SINGAPORE SPORTS SCHOOL,21,5,None,Shot put,None,7.26kg,None,...,2025-08-16 17:26:50.424539+00:00,463 days 17:26:50.424539,463,5,sheik alauddin sheik fayz,"[sheik, alauddin, sheik, fayz]","[2003-08-16 00:00:00, 2003-08-16 00:00:00]","[sheik alauddin sheik fayz, sheik alauddin fay...",2003-08-16,22.376454


In [2122]:
# Choose athletes at least 16 yrs old based on U18 minimum

athletes = athletes[(athletes['age_dec25']>=16) & (athletes['age_dec25']<=30)]

In [2123]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,NOW,delta_time,delta_time_conv,event_month,clean_name,name_to_list,dob_list,permutations,dob_first,age_dec25
0,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,2025-08-16 17:26:50.424539+00:00,27 days 17:26:50.424539,27,7,eric yee chun wai,"[eric, yee, chun, wai]","[1999-03-01 00:00:00, 1999-03-01 00:00:00, 199...","[eric yee chun wai, eric yee wai chun, eric ch...",1999-03-01,26.836413
2,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,2025-08-16 17:26:50.424539+00:00,48 days 17:26:50.424539,48,6,lucas wong yijie,"[lucas, wong, yijie]",[2004-05-05 00:00:00],"[lucas wong yijie, lucas yijie wong, wong luca...",2004-05-05,21.656400
24,Hui Ting Tan,10.50,Raffles Institution JC -,,1,,Shot Put,,3Kilogram,,...,2025-08-16 17:26:50.424539+00:00,21 days 02:26:50.424539,21,7,hui ting tan,"[hui, ting, tan]","[2008-02-18 00:00:00, 2008-02-18 00:00:00]","[hui ting tan, hui tan ting, ting hui tan, tin...",2008-02-18,17.867214
26,Kimmy Yu Tong Tay,8.25,Singapore Sports School -,,3,,Shot Put,,3Kilogram,,...,2025-08-16 17:26:50.424539+00:00,21 days 02:26:50.424539,21,7,kimmy yu tong tay,"[kimmy, yu, tong, tay]","[2009-01-02 00:00:00, 2009-02-01 00:00:00]","[kimmy yu tong tay, kimmy yu tay tong, kimmy t...",2009-01-02,16.993840
31,Anson loh,17.74,Fatties Association Throws -,,1,,Shot Put,,5Kilogram,,...,2025-08-16 17:26:50.424539+00:00,14 days 03:56:50.424539,14,8,anson loh,"[anson, loh]","[2008-08-16 00:00:00, 2008-04-28 00:00:00]","[anson loh, loh anson]",2008-08-16,17.374401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,"See, Kellen",11.21m,Victoria Junior College,17,2,U18,Shot Put,0.0,(5kg),nan,...,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,see kellen,"[see, kellen]",[2008-01-01 00:00:00],"[see kellen, kellen see]",2008-01-01,17.998631
1681,"Tang, Jet",14.15m,Victoria Junior College,17,1,U18,Shot Put,0.0,(5kg),nan,...,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,tang jet,"[tang, jet]",[2008-01-01 00:00:00],"[tang jet, jet tang]",2008-01-01,17.998631
1687,"Abdullah, Siti Annur Nadiera",10.45m,SSNTHO,17,1,U18,Shot Put,0.0,(3kg),nan,...,2025-08-16 17:26:50.424539+00:00,77 days 17:26:50.424539,77,5,abdullah siti annur nadiera,"[abdullah, siti, annur, nadiera]",[2008-01-01 00:00:00],"[abdullah siti annur nadiera, abdullah siti na...",2008-01-01,17.998631
1688,"SHEIK ALAUDDIN, SHEIK FAYZ",10.51,SINGAPORE SPORTS SCHOOL,21,5,None,Shot put,None,7.26kg,None,...,2025-08-16 17:26:50.424539+00:00,463 days 17:26:50.424539,463,5,sheik alauddin sheik fayz,"[sheik, alauddin, sheik, fayz]","[2003-08-16 00:00:00, 2003-08-16 00:00:00]","[sheik alauddin sheik fayz, sheik alauddin fay...",2003-08-16,22.376454


In [2124]:
# Create temporary mapped event column
# Junior Category Only

athletes['MAPPED_EVENT']=''

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()


# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'

# Standardise Open category for joining later

mask = (athletes['DIVISION'].str.contains(r'Open', na=False))
athletes.loc[mask, 'DIVISION'] = 'OPEN'

mask = (athletes['DIVISION'].str.contains(r'A Div', na=False))
athletes.loc[mask, 'DIVISION'] = 'A'


# Throws

# Javelin

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_CLASS'] = '600g'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_CLASS'] = '600g'


mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_CLASS'] = '800g'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_CLASS'] = '800g'


# Shot Put

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_CLASS'] = '7.26kg'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_CLASS'] = '7.26kg'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_CLASS'] = '4.00kg'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_CLASS'] = '4.00kg'


# Discus

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False)) # U18
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '2.00kg'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '2.00kg'


mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '1kg'


# Normalise EVENT_CLASS column

mask = (athletes['EVENT_CLASS'].str.contains(r'1.00kg|1kg|1Kilogram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '1kg'
mask = (athletes['EVENT_CLASS'].str.contains(r'1.5kg|1.50kg|1.5Kilogram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '1.50kg'
mask = (athletes['EVENT_CLASS'].str.contains(r'1.75kg|1.75Kilogram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '1.75kg'
mask = (athletes['EVENT_CLASS'].str.contains(r'2.00kg|2kg|2Kilogram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '2.00kg'
mask = (athletes['EVENT_CLASS'].str.contains(r'3.00kg|3kg|3Kilogram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '3.00kg'
mask = (athletes['EVENT_CLASS'].str.contains(r'4.00kg|4kg|4Kilogram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '4.00kg'
mask = (athletes['EVENT_CLASS'].str.contains(r'5.00kg|5kg|5Kilogram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '5.00kg'
mask = (athletes['EVENT_CLASS'].str.contains(r'6.00kg|6kg|6Kilogram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '6.00kg'
mask = (athletes['EVENT_CLASS'].str.contains(r'7.26kg|7.26Kilogram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '7.26kg'


mask = (athletes['EVENT_CLASS'].str.contains(r'500g|500Gram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '500g'
mask = (athletes['EVENT_CLASS'].str.contains(r'600g|600Gram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '600g'
mask = (athletes['EVENT_CLASS'].str.contains(r'700g|700Gram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '700g'
mask = (athletes['EVENT_CLASS'].str.contains(r'800g|800Gram', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '800g'

# Youth specific competitions to weight class (TO CHECK WITH AKID)

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['COMPETITION'].str.contains(r'Youth', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '1kg'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['COMPETITION'].str.contains(r'Youth', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '1.50kg'

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin|Javelin throw', na=False) & athletes['COMPETITION'].str.contains(r'Youth', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '500g'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin|Javelin throw', na=False) & athletes['COMPETITION'].str.contains(r'Youth', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '700g'

mask = (athletes['EVENT'].str.contains(r'Shot put|Shot Put', na=False) & athletes['COMPETITION'].str.contains(r'Youth', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '3.00kg'
mask = (athletes['EVENT'].str.contains(r'Shot put|Shot Put', na=False) & athletes['COMPETITION'].str.contains(r'Youth', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '5.00kg'

# The rest of the events (Note:U15 not mapped)

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '1kg'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '1.50kg'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False) 
        & athletes['DIVISION'].str.contains(r'U20', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '1kg'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False) 
        & athletes['DIVISION'].str.contains(r'U20', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '1.75kg'


mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin|Javelin throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '500g'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin|Javelin throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '700g'

mask = (athletes['EVENT'].str.contains(r'Shot put|Shot Put', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '3.00kg'
mask = (athletes['EVENT'].str.contains(r'Shot put|Shot Put', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False) 
        & athletes['DIVISION'].str.contains(r'U18', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'MAPPED_CLASS'] = '5.00kg'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/992498090.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['MAPPED_EVENT']=''
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/992498090.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/992498090.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [2125]:
# Imply Divison From Region, Age and Event_class
# U18 end of 31 Dec 2025 below 18 (16 to 17)
# U20 end of 31 Dec 2025 below 20 (18 to 19)
# Senior(OPEN) end of 31 Dec 2025 (>=16 yrs)


athletes['age_dec25'] = pd.to_numeric(athletes['age_dec25'], errors='coerce')

# Male Discus OPEN
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False)  & athletes['MAPPED_CLASS'].str.contains(r'2.00kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'DIVISION_MAPPED'] = 'OPEN'

mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Discus', na=False) & athletes['MAPPED_CLASS'].str.contains(r'2.00kg', na=False) 
        & athletes['GENDER'].str.contains(r'Male', na=False) & (athletes['age_dec25']>=16)) 
athletes.loc[mask, 'DIVISION_MAPPED'] = 'OPEN'


# Male Discus U20
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Discus', na=False) & athletes['MAPPED_CLASS'].str.contains(r'1.75kg', na=False) 
        & athletes['GENDER'].str.contains(r'Male', na=False) & ((athletes['age_dec25']>=18) & (athletes['age_dec25']<=19))) 
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U20'

# Male Discus U18
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Discus', na=False) & athletes['MAPPED_CLASS'].str.contains(r'1.50kg', na=False) 
        & athletes['GENDER'].str.contains(r'Male', na=False) & ((athletes['age_dec25']>=16) & (athletes['age_dec25']<=17))) 
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U18'

##########

# Female Discus OPEN
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False)  & athletes['MAPPED_CLASS'].str.contains(r'1kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'DIVISION_MAPPED'] = 'OPEN'

# Female Discus U18
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Discus', na=False) & athletes['MAPPED_CLASS'].str.contains(r'1kg', na=False) 
        & athletes['GENDER'].str.contains(r'Female', na=False) & ((athletes['age_dec25']>=16) & (athletes['age_dec25']<=17))) # U18
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U18'

# Female Discus U20
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Discus', na=False) & athletes['MAPPED_CLASS'].str.contains(r'1kg', na=False) 
        & athletes['GENDER'].str.contains(r'Female', na=False) & ((athletes['age_dec25']>=18) & (athletes['age_dec25']<=19))) 
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U20'

#########

# Female Shot Put U18
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Shot Put', na=False) & athletes['MAPPED_CLASS'].str.contains(r'3.00kg', na=False) 
        & athletes['GENDER'].str.contains(r'Female', na=False) & ((athletes['age_dec25']>=16) & (athletes['age_dec25']<=17))) # U18
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U18'

# Female Shot Put U20
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Shot Put', na=False) & athletes['MAPPED_CLASS'].str.contains(r'4.00kg', na=False) 
        & athletes['GENDER'].str.contains(r'Female', na=False) & ((athletes['age_dec25']>=18) & (athletes['age_dec25']<=19))) 
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U20'

# Female Shot Put OPEN (check due to overlap with U20)
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Shot Put', na=False) & athletes['MAPPED_CLASS'].str.contains(r'4.00kg', na=False) 
        & athletes['GENDER'].str.contains(r'Female', na=False) & (athletes['age_dec25']>=19)) 
athletes.loc[mask, 'DIVISION_MAPPED'] = 'OPEN'



###########

# Male Shot Put U18
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Shot Put', na=False) & athletes['MAPPED_CLASS'].str.contains(r'5.00kg', na=False) 
        & athletes['GENDER'].str.contains(r'Male', na=False) & ((athletes['age_dec25']>=16) & (athletes['age_dec25']<=17))) # U18
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U18'

# Male Shot Put U20
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Shot Put', na=False) & athletes['MAPPED_CLASS'].str.contains(r'6.00kg', na=False) 
        & athletes['GENDER'].str.contains(r'Male', na=False) & ((athletes['age_dec25']>=18) & (athletes['age_dec25']<=19))) 
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U20'

# Male Shot Put OPEN (check)
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & athletes['MAPPED_CLASS'].str.contains(r'7.26kg', na=False) 
        & athletes['GENDER'].str.contains(r'Male', na=False) & (athletes['age_dec25']>=16)) 
athletes.loc[mask, 'DIVISION_MAPPED'] = 'OPEN'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & athletes['REGION'].str.contains(r'International', na=False)  & athletes['MAPPED_CLASS'].str.contains(r'7.26kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'DIVISION_MAPPED'] = 'OPEN'


##########

# Male Javelin OPEN

mask = (athletes['EVENT'].str.contains(r'Javelin', na=False) & athletes['REGION'].str.contains(r'International', na=False)  & athletes['MAPPED_CLASS'].str.contains(r'800g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # U18
athletes.loc[mask, 'DIVISION_MAPPED'] = 'OPEN'

mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Javelin', na=False) & athletes['MAPPED_CLASS'].str.contains(r'800g', na=False) 
        & athletes['GENDER'].str.contains(r'Male', na=False) & (athletes['age_dec25']>=16)) # U18
athletes.loc[mask, 'DIVISION_MAPPED'] = 'OPEN'

# Male Javelin U18

mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Javelin', na=False) & athletes['MAPPED_CLASS'].str.contains(r'700g', na=False) 
        & athletes['GENDER'].str.contains(r'Male', na=False) & ((athletes['age_dec25']>=16) & (athletes['age_dec25']<=17))) # U18
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U18'

# Male Javelin U20
mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Javelin', na=False) & athletes['MAPPED_CLASS'].str.contains(r'700g', na=False) 
        & athletes['GENDER'].str.contains(r'Male', na=False) & ((athletes['age_dec25']>=18) & (athletes['age_dec25']<=19))) 
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U20'

###########

# Female Javelin OPEN

mask = (athletes['EVENT'].str.contains(r'Javelin', na=False) & athletes['REGION'].str.contains(r'International', na=False)  & athletes['MAPPED_CLASS'].str.contains(r'600g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'DIVISION_MAPPED'] = 'OPEN'

# Female Javelin U18

mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Javelin', na=False) & athletes['MAPPED_CLASS'].str.contains(r'500g', na=False) 
        & athletes['GENDER'].str.contains(r'Female', na=False) & ((athletes['age_dec25']>=16) & (athletes['age_dec25']<=17))) # U18
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U18'

# Female Javelin U20

mask = ((athletes['DIVISION'].isna() | (athletes['DIVISION'].str.strip() == '')) & athletes['EVENT'].str.contains(r'Javelin', na=False) & athletes['MAPPED_CLASS'].str.contains(r'600g', na=False) 
        & athletes['GENDER'].str.contains(r'Female', na=False) & ((athletes['age_dec25']>=18) & (athletes['age_dec25']<=19))) 
athletes.loc[mask, 'DIVISION_MAPPED'] = 'U20'




/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/4157780539.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['age_dec25'] = pd.to_numeric(athletes['age_dec25'], errors='coerce')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/4157780539.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes.loc[mask, 'DIVISION_MAPPED'] = 'OPEN'


In [2126]:


# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
        #    print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
                
                else:
    
                    output=float(str(metric))
        
            elif string=='':   # no event description at all!
                
                output='' # return nothing
            
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
                elif '10,000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

                elif '5000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

                    
                    
                elif '1500m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                    
                    if len(substring)==7:  # format is X:XX:XX and not XX:XX:XX 
                        
                        idx = 4 # 5th character position
                        replacement = "."
                        metric = '0' + metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                        
                    else:  # format is XX:XX:XX
                        
                        idx = 5 # 5th character position
                        replacement = "."
                        metric = metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())  
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output



In [2127]:
#Convert results and strip out FOUL and other results

athletes.reset_index(drop=True, inplace=True)

for col in athletes.columns:
    
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

for i in range(len(athletes)):
    
    result_out=''
    
        
    rowIndex = athletes.index[i]

    event=athletes.loc[rowIndex,'MAPPED_EVENT']    # event description
    
    result=athletes.loc[rowIndex,'RESULT'] # result
    
    if result=='—' or result=='None' or result=='DQ' or result=='SCR' or result=='FS' or result=='DNQ' or result=='DNS' or result=='NH' or result=='NM' or result=='FOUL' or result=='DNF' or result=='SR' :
        continue
    
    result_out = convert_time(i, event, result)
         
    athletes.loc[rowIndex, 'RESULT_CONV'] = result_out



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/2130528269.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/2130528269.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_53271/2130528269.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [2128]:
# Filter out records with no results and foreign teams

athletes=athletes[athletes['RESULT_CONV'].notnull()]

athletes = athletes[(athletes['TEAM']!='Sri Lanka')&(athletes['TEAM']!='SOUTH KOREA')&(athletes['TEAM']!='Vietnam')&(athletes['TEAM']!='THAILAND')
                    &(athletes['TEAM']!='CHINESE TAIPEI')&(athletes['TEAM']!='Indonesia')] 
athletes = athletes[(athletes['NATIONALITY']!='Chi')&(athletes['NATIONALITY']!='CHINA')&(athletes['NATIONALITY']!='GBR')&(athletes['NATIONALITY']!='INA')
                    &(athletes['NATIONALITY']!='IND')&(athletes['NATIONALITY']!='Ire')&(athletes['NATIONALITY']!='KOR')&(athletes['NATIONALITY']!='Mal')
                    &(athletes['NATIONALITY']!='USA')&(athletes['NATIONALITY']!='MAS')&(athletes['NATIONALITY']!='PHI')&(athletes['NATIONALITY']!='TPE')
                    &(athletes['NATIONALITY']!='TPE')&(athletes['NATIONALITY']!='CHN')&(athletes['NATIONALITY']!='Ind')&(athletes['NATIONALITY']!='JPN')] 



In [2129]:
# Use cublic spline to extrapolate

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

# Hardcoded mapping matrix
matrix = np.array([
    [30,   32.5, 35,   40],
    [32.5, 35,   37.5, 42.5],
    [35,   37.5, 40,   45],
    [37.5, 40,   42.5, 47.5],
    [40,   42.5, 45,   50],
    [42.5, 45,   47.5, 52.5],
    [45,   47.5, 50,   55],
    [47.5, 50,   52.5, 57.5],
    [50,   52.5, 55,   60],
    [52,   55,   57.5, 62.5],
    [54,   57.5, 60,   65],
    [56,   60,   62.5, 67.5],
    [58,   62.5, 65,   70],
    [60,   65,   67.5, 72.5],
    [62,   67.5, 70,   75],
    [64,   70,   72.5, 77.5],
], dtype=float)

# Map of weight → column index
weight_map = {2.0: 0, 1.75: 1, 1.5: 2, 1.25: 3}

# Function with cubic interpolation + extrapolation
def map_distance(from_weight, distance, to_weight):
    from_col = weight_map[from_weight]
    to_col = weight_map[to_weight]
    
    f = interp1d(
        matrix[:, from_col],
        matrix[:, to_col],
        kind="cubic",
        fill_value="extrapolate"   # allows extrapolation outside range
    )
    return float(f(distance))

# Apply only under conditions
def adjust_distance(row):
    if (
        row['MAPPED_EVENT'] in ['Discus', 'Discus Throw']
        and row['GENDER'] == 'Male'
        and row['COMPETITION'] == 'National School Games'
        and row['DIVISION'] == 'A'
        and row['MAPPED_CLASS'] == '1.50kg'
    ):
        return map_distance(1.5, row['RESULT_CONV'], 2.0)
    else:
        return np.nan

# Example: apply to dataframe
athletes['ADJUSTED_DISTANCE'] = athletes.apply(adjust_distance, axis=1)


In [2130]:
# If NSG Div A and Discus then choose adjusted_distance otherwise result_conv

condition1 = ((athletes['COMPETITION']=='National School Games') & (athletes['MAPPED_EVENT']=='Discus') & (athletes['DIVISION']=='A'))

athletes['RESULT_MAPPED'] = athletes['ADJUSTED_DISTANCE'].where((condition1), athletes['RESULT_CONV'].values)

athletes[['RESULT_MAPPED']] = athletes[['RESULT_MAPPED']].apply(pd.to_numeric)


In [2131]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Throws Program/')

athletes.to_csv('athletes_post_map_throws.csv', sep=',', encoding='utf-8-sig', index=False)


## Process Benchmarks

In [2132]:
# Load Benchmarks

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Throws Program/')

benchmarks = pd.read_csv("Benchmarks.csv")


In [2133]:
benchmarks['BENCHMARK'] = benchmarks['BENCHMARK'].astype('float64')

In [2134]:
benchmarks

,GENDER,SQUAD,EVENT_CLASS,DIVISION,EVENT,BENCHMARK
0,Male,NATIONAL,1.50kg,U18,Discus,47.64
1,Male,NATIONAL,1.75kg,U20,Discus,50.00
2,Male,NATIONAL,2.00kg,OPEN,Discus,50.02
3,Female,NATIONAL,1kg,U18,Discus,34.88
4,Female,NATIONAL,1kg,U20,Discus,45.08
5,Female,NATIONAL,1kg,OPEN,Discus,45.08
6,Male,NATIONAL,700g,U18,Javelin,58.99
7,Male,NATIONAL,800g,U20,Javelin,66.20
8,Male,NATIONAL,800g,OPEN,Javelin,66.20
9,Female,NATIONAL,500g,U18,Javelin,41.45


In [2136]:
mask = benchmarks['EVENT'].str.lower().str.contains(r'javelin|shot put|discus', na=True)

benchmarks.loc[mask, '2%']=benchmarks['BENCHMARK']*0.98
benchmarks.loc[mask, '3.5%']=benchmarks['BENCHMARK']*0.965
benchmarks.loc[mask, '5%']=benchmarks['BENCHMARK']*0.95
benchmarks.loc[mask, '10%']=benchmarks['BENCHMARK']*0.90
benchmarks.loc[mask, '15%']=benchmarks['BENCHMARK']*0.85
benchmarks.loc[mask, '20%']=benchmarks['BENCHMARK']*0.80


#benchmarks.loc[~mask, '2%']=benchmarks['Metric']*1.02
#benchmarks.loc[~mask, '3.5%']=benchmarks['Metric']*1.035
#benchmarks.loc[~mask, '5%']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'

# add 10%, 15% and 20%

In [2137]:
benchmarks

,GENDER,SQUAD,EVENT_CLASS,DIVISION,EVENT,BENCHMARK,2%,3.5%,5%,10%,15%,20%
0,Male,NATIONAL,1.50kg,U18,Discus,47.64,46.6872,45.97260,45.2580,42.876,40.4940,38.112
1,Male,NATIONAL,1.75kg,U20,Discus,50.00,49.0000,48.25000,47.5000,45.000,42.5000,40.000
2,Male,NATIONAL,2.00kg,OPEN,Discus,50.02,49.0196,48.26930,47.5190,45.018,42.5170,40.016
3,Female,NATIONAL,1kg,U18,Discus,34.88,34.1824,33.65920,33.1360,31.392,29.6480,27.904
4,Female,NATIONAL,1kg,U20,Discus,45.08,44.1784,43.50220,42.8260,40.572,38.3180,36.064
5,Female,NATIONAL,1kg,OPEN,Discus,45.08,44.1784,43.50220,42.8260,40.572,38.3180,36.064
6,Male,NATIONAL,700g,U18,Javelin,58.99,57.8102,56.92535,56.0405,53.091,50.1415,47.192
7,Male,NATIONAL,800g,U20,Javelin,66.20,64.8760,63.88300,62.8900,59.580,56.2700,52.960
8,Male,NATIONAL,800g,OPEN,Javelin,66.20,64.8760,63.88300,62.8900,59.580,56.2700,52.960
9,Female,NATIONAL,500g,U18,Javelin,41.45,40.6210,39.99925,39.3775,37.305,35.2325,33.160


In [2138]:
#benchmark['MAPPED_EVENT']=benchmark['EVENT'].str.strip()

In [2139]:
'''
for col in benchmark.columns:
    benchmark[col] = benchmark[col].astype(str)
    benchmark[col] = benchmark[col].str.replace('\xa0', ' ', regex=True)
    benchmark[col] = benchmark[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    benchmark[col] = benchmark[col].str.replace('\r', ' ', regex=True)
    benchmark[col] = benchmark[col].str.replace('\n', ' ', regex=True)
    benchmark[col] = benchmark[col].str.strip()
'''

"\nfor col in benchmark.columns:\n    benchmark[col] = benchmark[col].astype(str)\n    benchmark[col] = benchmark[col].str.replace('\xa0', ' ', regex=True)\n    benchmark[col] = benchmark[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)\n    benchmark[col] = benchmark[col].str.replace('\r', ' ', regex=True)\n    benchmark[col] = benchmark[col].str.replace('\n', ' ', regex=True)\n    benchmark[col] = benchmark[col].str.strip()\n"

In [2140]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [2141]:
# Select NATIONAL benchmarks and merge with performance benchmarks

benchmark_national=benchmarks[benchmarks['SQUAD']=='NATIONAL']

In [2142]:
benchmark_national

,GENDER,SQUAD,EVENT_CLASS,DIVISION,EVENT,BENCHMARK,2%,3.5%,5%,10%,15%,20%
0,Male,NATIONAL,1.50kg,U18,Discus,47.64,46.6872,45.97260,45.2580,42.876,40.4940,38.112
1,Male,NATIONAL,1.75kg,U20,Discus,50.00,49.0000,48.25000,47.5000,45.000,42.5000,40.000
2,Male,NATIONAL,2.00kg,OPEN,Discus,50.02,49.0196,48.26930,47.5190,45.018,42.5170,40.016
3,Female,NATIONAL,1kg,U18,Discus,34.88,34.1824,33.65920,33.1360,31.392,29.6480,27.904
4,Female,NATIONAL,1kg,U20,Discus,45.08,44.1784,43.50220,42.8260,40.572,38.3180,36.064
5,Female,NATIONAL,1kg,OPEN,Discus,45.08,44.1784,43.50220,42.8260,40.572,38.3180,36.064
6,Male,NATIONAL,700g,U18,Javelin,58.99,57.8102,56.92535,56.0405,53.091,50.1415,47.192
7,Male,NATIONAL,800g,U20,Javelin,66.20,64.8760,63.88300,62.8900,59.580,56.2700,52.960
8,Male,NATIONAL,800g,OPEN,Javelin,66.20,64.8760,63.88300,62.8900,59.580,56.2700,52.960
9,Female,NATIONAL,500g,U18,Javelin,41.45,40.6210,39.99925,39.3775,37.305,35.2325,33.160


## Left Join Benchmarks onto Results Table

In [2143]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmark_national,
    how='left',
    left_on=['GENDER', 'MAPPED_EVENT', 'DIVISION','MAPPED_CLASS'],
    right_on=['GENDER', 'EVENT', 'DIVISION', 'EVENT_CLASS'],
)

In [2144]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS_x,UNIQUE_ID,...,SQUAD,EVENT_CLASS_y,EVENT_y,BENCHMARK,2%,3.5%,5%,10%,15%,20%
0,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hui Ting Tan,10.50,Raffles Institution JC -,,1,,Shot Put,,3Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kimmy Yu Tong Tay,8.25,Singapore Sports School -,,3,,Shot Put,,3Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Anson loh,17.74,Fatties Association Throws -,,1,,Shot Put,,5Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,"See, Kellen",11.21m,Victoria Junior College,17,2,U18,Shot Put,0.0,(5kg),nan,...,NATIONAL,5.00kg,Shot Put,15.28,14.9744,14.74520,14.5160,13.752,12.9880,12.224
1021,"Tang, Jet",14.15m,Victoria Junior College,17,1,U18,Shot Put,0.0,(5kg),nan,...,NATIONAL,5.00kg,Shot Put,15.28,14.9744,14.74520,14.5160,13.752,12.9880,12.224
1022,"Abdullah, Siti Annur Nadiera",10.45m,SSNTHO,17,1,U18,Shot Put,0.0,(3kg),nan,...,NATIONAL,3.00kg,Shot Put,12.91,12.6518,12.45815,12.2645,11.619,10.9735,10.328
1023,"SHEIK ALAUDDIN, SHEIK FAYZ",10.51,SINGAPORE SPORTS SCHOOL,21,5,None,Shot put,None,7.26kg,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2145]:
# Change to numeric

df[['2%', '3.5%', '5%', '10%', '15%', '20%', 'RESULT_MAPPED', 'BENCHMARK']] = df[['2%', '3.5%', '5%', '10%', '15%', '20%', 'RESULT_MAPPED', 'BENCHMARK']].apply(pd.to_numeric, errors='coerce')

In [2146]:
mask = df['MAPPED_EVENT'].str.lower().str.contains(r'javelin|shot put|discus', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_MAPPED']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_MAPPED']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_MAPPED']-df['5%']
df.loc[mask, 'Delta10'] = df['RESULT_MAPPED']-df['10%']
df.loc[mask, 'Delta15'] = df['RESULT_MAPPED']-df['15%']
df.loc[mask, 'Delta20'] = df['RESULT_MAPPED']-df['20%']

df.loc[mask, 'Delta_Benchmark'] = df['RESULT_MAPPED']-df['BENCHMARK']

#df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_BEST']
#df.loc[~mask, 'Delta3.5'] = df['3.5%'] - df['RESULT_BEST']
#df.loc[~mask, 'Delta5'] = df['5%'] - df['RESULT_BEST']
#df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_BEST']


In [2147]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS_x,UNIQUE_ID,...,10%,15%,20%,Delta2,Delta3.5,Delta5,Delta10,Delta15,Delta20,Delta_Benchmark
0,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hui Ting Tan,10.50,Raffles Institution JC -,,1,,Shot Put,,3Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kimmy Yu Tong Tay,8.25,Singapore Sports School -,,3,,Shot Put,,3Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Anson loh,17.74,Fatties Association Throws -,,1,,Shot Put,,5Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,"See, Kellen",11.21m,Victoria Junior College,17,2,U18,Shot Put,0.0,(5kg),nan,...,13.752,12.9880,12.224,-3.7644,-3.53520,-3.3060,-2.542,-1.7780,-1.014,-4.07
1021,"Tang, Jet",14.15m,Victoria Junior College,17,1,U18,Shot Put,0.0,(5kg),nan,...,13.752,12.9880,12.224,-0.8244,-0.59520,-0.3660,0.398,1.1620,1.926,-1.13
1022,"Abdullah, Siti Annur Nadiera",10.45m,SSNTHO,17,1,U18,Shot Put,0.0,(3kg),nan,...,11.619,10.9735,10.328,-2.2018,-2.00815,-1.8145,-1.169,-0.5235,0.122,-2.46
1023,"SHEIK ALAUDDIN, SHEIK FAYZ",10.51,SINGAPORE SPORTS SCHOOL,21,5,None,Shot put,None,7.26kg,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2148]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta5']/df['BENCHMARK']*100

In [2149]:
df['TIER'] = np.where((df['Delta_Benchmark']>=0), 'Tier 1',    
             np.where(((df['Delta_Benchmark']<0) & (df['Delta2']>=0)), 'Tier 2',
             np.where(((df['Delta2']<0) & (df['Delta3.5']>=0)), 'Tier 3', 
             np.where(((df['Delta3.5']<0) & (df['Delta5']>=0)), 'Tier 4', ' '))))


In [2150]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS_x,UNIQUE_ID,...,20%,Delta2,Delta3.5,Delta5,Delta10,Delta15,Delta20,Delta_Benchmark,PERF_SCALAR,TIER
0,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,Hui Ting Tan,10.50,Raffles Institution JC -,,1,,Shot Put,,3Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,Kimmy Yu Tong Tay,8.25,Singapore Sports School -,,3,,Shot Put,,3Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,Anson loh,17.74,Fatties Association Throws -,,1,,Shot Put,,5Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,"See, Kellen",11.21m,Victoria Junior College,17,2,U18,Shot Put,0.0,(5kg),nan,...,12.224,-3.7644,-3.53520,-3.3060,-2.542,-1.7780,-1.014,-4.07,-21.636126,
1021,"Tang, Jet",14.15m,Victoria Junior College,17,1,U18,Shot Put,0.0,(5kg),nan,...,12.224,-0.8244,-0.59520,-0.3660,0.398,1.1620,1.926,-1.13,-2.395288,
1022,"Abdullah, Siti Annur Nadiera",10.45m,SSNTHO,17,1,U18,Shot Put,0.0,(3kg),nan,...,10.328,-2.2018,-2.00815,-1.8145,-1.169,-0.5235,0.122,-2.46,-14.054996,
1023,"SHEIK ALAUDDIN, SHEIK FAYZ",10.51,SINGAPORE SPORTS SCHOOL,21,5,None,Shot put,None,7.26kg,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [2151]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Throws Program/')

df.to_csv('Throws_postmap_benchmarked.csv', sep=',', encoding='utf-8-sig', index=False)


In [2152]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS_x,UNIQUE_ID,...,20%,Delta2,Delta3.5,Delta5,Delta10,Delta15,Delta20,Delta_Benchmark,PERF_SCALAR,TIER
0,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,Hui Ting Tan,10.50,Raffles Institution JC -,,1,,Shot Put,,3Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,Kimmy Yu Tong Tay,8.25,Singapore Sports School -,,3,,Shot Put,,3Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,Anson loh,17.74,Fatties Association Throws -,,1,,Shot Put,,5Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,"See, Kellen",11.21m,Victoria Junior College,17,2,U18,Shot Put,0.0,(5kg),nan,...,12.224,-3.7644,-3.53520,-3.3060,-2.542,-1.7780,-1.014,-4.07,-21.636126,
1021,"Tang, Jet",14.15m,Victoria Junior College,17,1,U18,Shot Put,0.0,(5kg),nan,...,12.224,-0.8244,-0.59520,-0.3660,0.398,1.1620,1.926,-1.13,-2.395288,
1022,"Abdullah, Siti Annur Nadiera",10.45m,SSNTHO,17,1,U18,Shot Put,0.0,(3kg),nan,...,10.328,-2.2018,-2.00815,-1.8145,-1.169,-0.5235,0.122,-2.46,-14.054996,
1023,"SHEIK ALAUDDIN, SHEIK FAYZ",10.51,SINGAPORE SPORTS SCHOOL,21,5,None,Shot put,None,7.26kg,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


## Adjust Discus Throw Distances for NSG Div A and 1.50kg

In [2153]:
'''
# Interpolate and Map Discus Throw Distances

import numpy as np
import pandas as pd

# Distance matrix
data = np.array([
    [30,   32.5, 35,   40],
    [32.5, 35,   37.5, 42.5],
    [35,   37.5, 40,   45],
    [37.5, 40,   42.5, 47.5],
    [40,   42.5, 45,   50],
    [42.5, 45,   47.5, 52.5],
    [45,   47.5, 50,   55],
    [47.5, 50,   52.5, 57.5],
    [50,   52.5, 55,   60],
    [52,   55,   57.5, 62.5],
    [54,   57.5, 60,   65],
    [56,   60,   62.5, 67.5],
    [58,   62.5, 65,   70],
    [60,   65,   67.5, 72.5],
    [62,   67.5, 70,   75],
    [64,   70,   72.5, 77.5]
])

# Labels
weights = ["2kg", "1.75kg", "1.5kg", "1.25kg"]
df = pd.DataFrame(data, columns=weights)

def map_distance(value, from_weight, to_weight):
    """
    Maps a throw distance from one discus weight to another using interpolation.
    """
    # Ensure valid weights
    if from_weight not in df.columns or to_weight not in df.columns:
        raise ValueError("Invalid weight. Choose from: " + ", ".join(df.columns))
    
    # Use numpy interpolation across the from_weight column
    from_vals = df[from_weight].values
    to_vals = df[to_weight].values
    
    # Interpolate (linear between points)
    mapped_value = np.interp(value, from_vals, to_vals)
    
    return mapped_value

# --- Example usage ---
throw_2kg = 36
mapped_to_1_5kg = map_distance(throw_2kg, "2kg", "1.5kg")
mapped_to_1_25kg = map_distance(throw_2kg, "2kg", "1.25kg")

print(f"{throw_2kg}m (2kg discus) ≈ {mapped_to_1_5kg:.2f}m (1.5kg discus)")
print(f"{throw_2kg}m (2kg discus) ≈ {mapped_to_1_25kg:.2f}m (1.25kg discus)")
'''

'\n# Interpolate and Map Discus Throw Distances\n\nimport numpy as np\nimport pandas as pd\n\n# Distance matrix\ndata = np.array([\n    [30,   32.5, 35,   40],\n    [32.5, 35,   37.5, 42.5],\n    [35,   37.5, 40,   45],\n    [37.5, 40,   42.5, 47.5],\n    [40,   42.5, 45,   50],\n    [42.5, 45,   47.5, 52.5],\n    [45,   47.5, 50,   55],\n    [47.5, 50,   52.5, 57.5],\n    [50,   52.5, 55,   60],\n    [52,   55,   57.5, 62.5],\n    [54,   57.5, 60,   65],\n    [56,   60,   62.5, 67.5],\n    [58,   62.5, 65,   70],\n    [60,   65,   67.5, 72.5],\n    [62,   67.5, 70,   75],\n    [64,   70,   72.5, 77.5]\n])\n\n# Labels\nweights = ["2kg", "1.75kg", "1.5kg", "1.25kg"]\ndf = pd.DataFrame(data, columns=weights)\n\ndef map_distance(value, from_weight, to_weight):\n    """\n    Maps a throw distance from one discus weight to another using interpolation.\n    """\n    # Ensure valid weights\n    if from_weight not in df.columns or to_weight not in df.columns:\n        raise ValueError("Invalid

In [2154]:
'''

# Does not handle out of range extrapolations so a 28.2 1.5kg throw will produce spurious results

import numpy as np

# Define mapping function (from earlier)
def map_distance(from_weight, distance, to_weight):
    # distance matrix (rows=base distances, cols=weights [2.0,1.75,1.5,1.25])
    base_distances = np.array([30, 32.5, 35, 37.5, 40, 42.5, 45, 47.5, 50, 52, 54, 56, 58, 60, 62, 64], dtype=float)
    weight_map = {2.0: 0, 1.75: 1, 1.5: 2, 1.25: 3}

    matrix = np.array([
        [30,   32.5, 35,   40],
        [32.5, 35,   37.5, 42.5],
        [35,   37.5, 40,   45],
        [37.5, 40,   42.5, 47.5],
        [40,   42.5, 45,   50],
        [42.5, 45,   47.5, 52.5],
        [45,   47.5, 50,   55],
        [47.5, 50,   52.5, 57.5],
        [50,   52.5, 55,   60],
        [52,   55,   57.5, 62.5],
        [54,   57.5, 60,   65],
        [56,   60,   62.5, 67.5],
        [58,   62.5, 65,   70],
        [60,   65,   67.5, 72.5],
        [62,   67.5, 70,   75],
        [64,   70,   72.5, 77.5],
    ], dtype=float)

    from_col = weight_map[from_weight]
    to_col = weight_map[to_weight]

    # interpolate along base_distances
    return np.interp(distance, matrix[:, from_col], matrix[:, to_col])

# Apply only under the specified conditions
def adjust_distance(row):
    if (
        row['MAPPED_EVENT'] in ['Discus', 'Discus Throw']
        and row['GENDER'] == 'Male'
        and row['COMPETITION'] == 'National School Games'
        and row['DIVISION'] == 'A'
        and row['MAPPED_CLASS'] == '1.50kg'
    ):
        return map_distance(1.5, row['RESULT_CONV'], 2.0)
    else:
        return np.nan  # leave blank if not applicable

df['ADJUSTED_DISTANCE'] = df.apply(adjust_distance, axis=1)
'''

"\n\n# Does not handle out of range extrapolations so a 28.2 1.5kg throw will produce spurious results\n\nimport numpy as np\n\n# Define mapping function (from earlier)\ndef map_distance(from_weight, distance, to_weight):\n    # distance matrix (rows=base distances, cols=weights [2.0,1.75,1.5,1.25])\n    base_distances = np.array([30, 32.5, 35, 37.5, 40, 42.5, 45, 47.5, 50, 52, 54, 56, 58, 60, 62, 64], dtype=float)\n    weight_map = {2.0: 0, 1.75: 1, 1.5: 2, 1.25: 3}\n\n    matrix = np.array([\n        [30,   32.5, 35,   40],\n        [32.5, 35,   37.5, 42.5],\n        [35,   37.5, 40,   45],\n        [37.5, 40,   42.5, 47.5],\n        [40,   42.5, 45,   50],\n        [42.5, 45,   47.5, 52.5],\n        [45,   47.5, 50,   55],\n        [47.5, 50,   52.5, 57.5],\n        [50,   52.5, 55,   60],\n        [52,   55,   57.5, 62.5],\n        [54,   57.5, 60,   65],\n        [56,   60,   62.5, 67.5],\n        [58,   62.5, 65,   70],\n        [60,   65,   67.5, 72.5],\n        [62,   67.5, 70,

In [2155]:
'''
# Use cublic spline to extrapolate

import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

# Hardcoded mapping matrix
matrix = np.array([
    [30,   32.5, 35,   40],
    [32.5, 35,   37.5, 42.5],
    [35,   37.5, 40,   45],
    [37.5, 40,   42.5, 47.5],
    [40,   42.5, 45,   50],
    [42.5, 45,   47.5, 52.5],
    [45,   47.5, 50,   55],
    [47.5, 50,   52.5, 57.5],
    [50,   52.5, 55,   60],
    [52,   55,   57.5, 62.5],
    [54,   57.5, 60,   65],
    [56,   60,   62.5, 67.5],
    [58,   62.5, 65,   70],
    [60,   65,   67.5, 72.5],
    [62,   67.5, 70,   75],
    [64,   70,   72.5, 77.5],
], dtype=float)

# Map of weight → column index
weight_map = {2.0: 0, 1.75: 1, 1.5: 2, 1.25: 3}

# Function with cubic interpolation + extrapolation
def map_distance(from_weight, distance, to_weight):
    from_col = weight_map[from_weight]
    to_col = weight_map[to_weight]
    
    f = interp1d(
        matrix[:, from_col],
        matrix[:, to_col],
        kind="cubic",
        fill_value="extrapolate"   # allows extrapolation outside range
    )
    return float(f(distance))

# Apply only under conditions
def adjust_distance(row):
    if (
        row['MAPPED_EVENT'] in ['Discus', 'Discus Throw']
        and row['GENDER'] == 'Male'
        and row['COMPETITION'] == 'National School Games'
        and row['DIVISION'] == 'A'
        and row['MAPPED_CLASS'] == '1.50kg'
    ):
        return map_distance(1.5, row['RESULT_CONV'], 2.0)
    else:
        return np.nan

# Example: apply to dataframe
df['ADJUSTED_DISTANCE'] = df.apply(adjust_distance, axis=1)
'''

'\n# Use cublic spline to extrapolate\n\nimport numpy as np\nimport pandas as pd\nfrom scipy.interpolate import interp1d\n\n# Hardcoded mapping matrix\nmatrix = np.array([\n    [30,   32.5, 35,   40],\n    [32.5, 35,   37.5, 42.5],\n    [35,   37.5, 40,   45],\n    [37.5, 40,   42.5, 47.5],\n    [40,   42.5, 45,   50],\n    [42.5, 45,   47.5, 52.5],\n    [45,   47.5, 50,   55],\n    [47.5, 50,   52.5, 57.5],\n    [50,   52.5, 55,   60],\n    [52,   55,   57.5, 62.5],\n    [54,   57.5, 60,   65],\n    [56,   60,   62.5, 67.5],\n    [58,   62.5, 65,   70],\n    [60,   65,   67.5, 72.5],\n    [62,   67.5, 70,   75],\n    [64,   70,   72.5, 77.5],\n], dtype=float)\n\n# Map of weight → column index\nweight_map = {2.0: 0, 1.75: 1, 1.5: 2, 1.25: 3}\n\n# Function with cubic interpolation + extrapolation\ndef map_distance(from_weight, distance, to_weight):\n    from_col = weight_map[from_weight]\n    to_col = weight_map[to_weight]\n    \n    f = interp1d(\n        matrix[:, from_col],\n        

In [2156]:
'''
# If NSG Div A and Discus then choose adjusted_distance otherwise result_conv

condition1 = ((df['COMPETITION']=='National School Games') & (df['MAPPED_EVENT']=='Discus') & (df['DIVISION']=='A'))

df['RESULT_MAPPED'] = df['ADJUSTED_DISTANCE'].where((condition1), athletes['RESULT_CONV'].values)

df[['RESULT_MAPPED']] = df[['RESULT_MAPPED']].apply(pd.to_numeric)
'''

"\n# If NSG Div A and Discus then choose adjusted_distance otherwise result_conv\n\ncondition1 = ((df['COMPETITION']=='National School Games') & (df['MAPPED_EVENT']=='Discus') & (df['DIVISION']=='A'))\n\ndf['RESULT_MAPPED'] = df['ADJUSTED_DISTANCE'].where((condition1), athletes['RESULT_CONV'].values)\n\ndf[['RESULT_MAPPED']] = df[['RESULT_MAPPED']].apply(pd.to_numeric)\n"

In [2157]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS_x,UNIQUE_ID,...,20%,Delta2,Delta3.5,Delta5,Delta10,Delta15,Delta20,Delta_Benchmark,PERF_SCALAR,TIER
0,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,Hui Ting Tan,10.50,Raffles Institution JC -,,1,,Shot Put,,3Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,Kimmy Yu Tong Tay,8.25,Singapore Sports School -,,3,,Shot Put,,3Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,Anson loh,17.74,Fatties Association Throws -,,1,,Shot Put,,5Kilogram,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,"See, Kellen",11.21m,Victoria Junior College,17,2,U18,Shot Put,0.0,(5kg),nan,...,12.224,-3.7644,-3.53520,-3.3060,-2.542,-1.7780,-1.014,-4.07,-21.636126,
1021,"Tang, Jet",14.15m,Victoria Junior College,17,1,U18,Shot Put,0.0,(5kg),nan,...,12.224,-0.8244,-0.59520,-0.3660,0.398,1.1620,1.926,-1.13,-2.395288,
1022,"Abdullah, Siti Annur Nadiera",10.45m,SSNTHO,17,1,U18,Shot Put,0.0,(3kg),nan,...,10.328,-2.2018,-2.00815,-1.8145,-1.169,-0.5235,0.122,-2.46,-14.054996,
1023,"SHEIK ALAUDDIN, SHEIK FAYZ",10.51,SINGAPORE SPORTS SCHOOL,21,5,None,Shot put,None,7.26kg,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [2158]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Throws Program/')

df.to_csv('check.csv', sep=',', encoding='utf-8-sig', index=False)


## Project Individial Results

In [2159]:
# Project results using age and np.array

athletes[['age_dec25', 'RESULT_MAPPED']] = athletes[['age_dec25', 'RESULT_MAPPED']].apply(pd.to_numeric, errors='coerce')

# Hardcoded projection matrix with >=4 decimal precision
ages = np.arange(16, 35, 1)  # ages 16 to 34

#factors = np.array([
#    1.0304, 1.0297, 1.0270, 1.0337, 1.050, 
#    1.0373, 1.0264, 1.0102, 1.0048, 1.0104, 
#    1.0056, 1.0066, 1.0076, 1.0014, 0.9979, 
#    0.9827, 1.0042, 1.0019, 0.9945
#])

factors = np.array([
    1.0000, 1.0200, 1.0500, 1.0800, 1.1000, 
    1.1200, 1.1400, 1.1500, 1.1700, 1.1800, 
    1.1900, 1.2000, 1.2100, 1.2200, 1.2300, 
    1.2400, 1.2450, 1.2470, 1.2500
])

def project_athlete(age, distance, ages, factors):
    """Project distances for an athlete given age and base throw."""
    # Get the scaling factor at athlete's base age (interpolated)
    base_factor = np.interp(age, ages, factors)
    relative_factors = factors / base_factor
    projected = distance * relative_factors
    # Remove ages before current
    projected = np.where(ages < age, np.nan, projected)
    return projected

# Apply function row by row
athletes["PROJECTIONS"] = athletes.apply(
    lambda row: project_athlete(row["age_dec25"], row["RESULT_CONV"], ages, factors),
    axis=1
)

# Expand PROJECTIONS into separate columns
proj_df = pd.DataFrame(athletes["PROJECTIONS"].tolist(), columns=[f"AGE_{a}" for a in ages])

# Merge back to original
df_projection = pd.concat([athletes.drop(columns=["PROJECTIONS"]), proj_df], axis=1)


In [2160]:
df_projection

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,AGE_25,AGE_26,AGE_27,AGE_28,AGE_29,AGE_30,AGE_31,AGE_32,AGE_33,AGE_34
0,Eric Yee Chun Wai,48.04,,,1,,Discus Throw,,,,...,NaN,NaN,48.105579,48.506458,48.907338,49.308218,49.709098,49.909538,49.989714,50.109978
1,Lucas Wong Yijie,56.37,,,,,Discus Throw,,,,...,58.701753,59.199226,59.696698,60.194171,60.691643,61.189116,61.686588,61.935324,62.034819,62.184061
2,Hui Ting Tan,10.50,Raffles Institution JC -,,1,,Shot Put,,3Kilogram,,...,11.844938,11.945319,12.045700,12.146081,12.246462,12.346842,12.447223,12.497414,12.517490,12.547604
3,Kimmy Yu Tong Tay,8.25,Singapore Sports School -,,3,,Shot Put,,3Kilogram,,...,9.545271,9.626163,9.707055,9.787947,9.868839,9.949731,10.030623,10.071069,10.087248,10.111515
4,Anson loh,17.74,Fatties Association Throws -,,1,,Shot Put,,5Kilogram,,...,20.299214,20.471242,20.643269,20.815296,20.987323,21.159350,21.331378,21.417391,21.451797,21.503405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.139070,8.208045,8.277020,8.345995,8.414970,8.483945,8.552921,8.587408,8.601203,8.621896
914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.248066,8.317965,8.387864,8.457763,8.527661,8.597560,8.667459,8.702409,8.716388,8.737358
915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.582654,12.689287,12.795920,12.902552,13.009185,13.115818,13.222450,13.275767,13.297093,13.329083
918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.614881,8.687889,8.760896,8.833903,8.906911,8.979918,9.052926,9.089430,9.104031,9.125933


In [2161]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Throws Program/')

df_projection.to_csv('projection.csv', sep=',', encoding='utf-8-sig', index=False)
